# Minimal Example

This notebook demonstrates the most concise way to create and optimize an energy system with **flixopt**.

It's perfect for:
- Quick prototyping
- Bug reports and feature proposals
- Learning the basic API structure

## Setup

First, import the required libraries:

In [ ]:
import numpy as np
import pandas as pd

import flixopt as fx

Configure flixopt for notebook environment (inline plots, reduced logging):

In [ ]:
fx.CONFIG.notebook()

## Create the FlowSystem

A `FlowSystem` is the container for all components. It requires a time index defining the optimization horizon.

Here we create a simple 3-hour horizon:

In [ ]:
flow_system = fx.FlowSystem(pd.date_range('2020-01-01', periods=3, freq='h'))

## Define Components

In this minimal example, we define:

1. **Buses** - Nodes where energy carriers are balanced (inputs = outputs)
2. **Effects** - Quantities to track/optimize (e.g., costs, emissions)
3. **Components** - Converters, sources, sinks that connect buses

All components can be added in a single `add_elements()` call:

In [ ]:
flow_system.add_elements(
    # Buses for heat and gas
    fx.Bus('Heat'),
    fx.Bus('Gas'),
    # Cost effect - this is what we minimize
    fx.Effect('Costs', '€', 'Cost', is_standard=True, is_objective=True),
    # A simple gas boiler (50% efficiency)
    fx.linear_converters.Boiler(
        'Boiler',
        thermal_efficiency=0.5,
        thermal_flow=fx.Flow(label='Heat', bus='Heat', size=50),
        fuel_flow=fx.Flow(label='Gas', bus='Gas'),
    ),
    # Heat demand (sink) with a fixed profile
    fx.Sink(
        'Sink',
        inputs=[fx.Flow(label='Demand', bus='Heat', size=1, fixed_relative_profile=np.array([30, 0, 20]))],
    ),
    # Gas supply (source) with cost of 0.04 €/kWh
    fx.Source(
        'Source',
        outputs=[fx.Flow(label='Gas', bus='Gas', size=1000, effects_per_flow_hour=0.04)],
    ),
)

## Run the Optimization

Now we solve the system using the HiGHS solver (included with flixopt):

In [ ]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0.01, time_limit_seconds=60))

## Analyze Results

The `statistics` accessor provides convenient methods for analyzing and plotting results.

Let's visualize the heat balance:

In [ ]:
flow_system.statistics.plot.balance('Heat')

## Summary

This minimal example shows the core workflow:

1. Create a `FlowSystem` with timesteps
2. Add buses, effects, and components
3. Call `optimize()`
4. Analyze results via `statistics`

For more complex examples, see the other notebooks in this series!